# Event Picker

This notebook allows to inspect the results from the clustering and estimation phases of Spyral, by displaying the locations of the tracks on the kinematics and PID plots, and showing the point cloud with the cluster determination. The clustering phase groups points from the point cloud into clusters, and the estimation phase determines the kinematical and energy loss for each of these clusters. If the estimation phase fails for a particular cluster, there is no corresponding track data recorded and the cluster is simply dropped. This explains why there can be more clusters than tracks.

When clicking somewhere on the kinematics or the PID plot, the closest data point of a particular track is picked and the event to which it belongs is displayed, as well as the data from other tracks of that event if there are any. The color scheme is consistent between all the plots, which helps in identifying which particles are involved in a given event.

This version handles only one run that has to be specified. The workspace path where all the Spyral files are located also needs to be specified.

In [ ]:
from spyral.core.point_cloud import PointCloud
from spyral.core.cluster import Cluster

import h5py as h5
import polars as pl
import numpy as np
from pathlib import Path
import matplotlib.pyplot as plt

%matplotlib widget
plt.close()

workspace_path = Path("/Enter/Your/Workspace/Path/Here")
cloud_path = workspace_path / "Pointcloud"
cluster_path = workspace_path / "Cluster"
estimation_result_path = workspace_path / "Estimation"
df = None
run = 123 # Your run number
path = estimation_result_path / f"run_{run:04d}.parquet"
cloud_file_path = cloud_path / f"run_{run:04d}.h5"
cloud_file = h5.File(cloud_file_path, 'r')
cloud_group: h5.Group = cloud_file.get('cloud')
cluster_file_path = cluster_path / f"run_{run:04d}.h5"
cluster_file = h5.File(cluster_file_path, 'r')
cluster_group: h5.Group = cluster_file.get('cluster')

df = pl.scan_parquet(path)
df = df.collect()


In [ ]:
class PointBrowser:

    def __init__(self):
        self.lastind = 0

        self.text = axs["C"].text(0.05, 0.95, 'selected event: none', transform=axs["C"].transAxes, va='top')
        self.tracks = axs["C"].text(0.05, 0.85, 'number of tracks: 0', transform=axs["C"].transAxes, va='top')
        self.clusters = axs["C"].text(0.05, 0.90, 'number of clusters: 0', transform=axs["C"].transAxes, va='top')
        self.marker1, = [axs["C"].plot(xs[0], ys[0], 'o', ms=5, alpha=0.7, color=cmap(0.0), visible=False)]
        self.marker2, = [axs["D"].plot(xs[0], ys[0], 'o', ms=5, alpha=0.7, color=cmap(0.0), visible=False)]
        for i in range(1,10):
            art, = axs["C"].plot(xs[i], ys[i], 'o', ms=5, alpha=0.7, color=cmap(float(i)/9.0), visible=False)
            self.marker1.append(art)
            art, = axs["D"].plot(xs[i], ys[i], 'o', ms=5, alpha=0.7, color=cmap(float(i)/9.0), visible=False)
            self.marker2.append(art)

    def on_pick(self, event):

        if event.artist != line and event.artist != line2:
            return True

        N = len(event.ind)
        if not N:
            return True

        # the click locations
        x = event.mouseevent.xdata
        y = event.mouseevent.ydata

        if event.artist == line:
            distances = np.hypot(x - xs[event.ind], y - ys[event.ind])
        elif event.artist == line2:
            distances = np.hypot(x - de[event.ind], y - ys[event.ind])
        indmin = distances.argmin()
        dataind = event.ind[indmin]

        self.lastind = dataind
        self.update()

    def update(self):
        if self.lastind is None:
            return

        dataind = self.lastind
        event = ev[dataind]
        self.update_plots(event)

    def update_plots(self, event):
        event_data = cloud_group[f'cloud_{event}']
        cloud = PointCloud(event, event_data[:].copy())

        axs["A"].clear()
        axs["B"].clear()
        axs["A"].scatter(cloud.data[:, 2], cloud.data[:, 0], cloud.data[:, 1], c=cloud.data[:, 3], s=1, label="Pointcloud")
        axs["A"].set_xlim3d(0., 1000.0)
        axs["A"].set_ylim3d(-300.0, 300.0)
        axs["A"].set_zlim3d(-300.0, 300.0)
        axs["B"].scatter(cloud.data[:, 0], cloud.data[:, 1], c=cloud.data[:, 3], s=1, label="Pointcloud")
        axs["B"].set_xlim(-300.0, 300.0)
        axs["B"].set_ylim(-300.0, 300.0)
    
        self.text.set_text('selected event: %d' % event)
        tracks = (ev == event).sum()
        self.tracks.set_text('number of tracks: %d' % tracks)
        ind, = np.where(ev == event)
        for i in range(tracks):
            self.marker1[i].set_data([xs[ind[i]]], [ys[ind[i]]])
            self.marker1[i].set_visible(True)
            self.marker1[i].set_color(cmap(float(cl[ind[i]])/9.0))
            self.marker1[i].set_label(f"Track {i}")
            self.marker2[i].set_data([de[ind[i]]], [ys[ind[i]]])
            self.marker2[i].set_color(cmap(float(cl[ind[i]])/9.0))
            self.marker2[i].set_visible(True)
            self.marker2[i].set_label(f"Track {i}")
        for i in range(tracks,10):
            self.marker1[i].set_visible(False)
            self.marker2[i].set_visible(False)
            self.marker1[i].set_label(f"_Track {i}")
            self.marker2[i].set_label(f"_Track {i}")
        axs["C"].legend()
        axs["D"].legend()

        clusters = cluster_group[f'event_{event}']
        nclusters = clusters.attrs["nclusters"]
        self.clusters.set_text('number of clusters: %d' % nclusters)
        for i in range(nclusters):
            cluster = clusters[f'cluster_{i}']
            cluster_cloud = cluster[f'cloud']
            cluster_data = Cluster(event, i, cluster_cloud[:].copy())
            axs["A"].scatter(cluster_data.data[:, 2], cluster_data.data[:, 0], cluster_data.data[:, 1], color=cmap(float(i)/9.0), s=3, label=f"Cluster {cluster_data.label}")
            axs["B"].scatter(cluster_data.data[:, 0], cluster_data.data[:, 1], color=cmap(float(i)/9.0), s=3, label=f"Cluster {cluster_data.label}")
        axs["A"].legend()
        axs["B"].legend()

        fig.canvas.draw()

plt.close()
cols = df.columns
# print(cols)
ev = np.array(df['event'])
ys = np.array(df['brho'])
xs = np.array(df["polar"])
de = np.array(df["sqrt_dEdx"])
cl = np.array(df["cluster_label"])

cmap = plt.cm.Set1

fig, axs = plt.subplot_mosaic(
    """
    CD
    AB
    """,
    per_subplot_kw={
        "A": {
            "projection": "3d", 
            "box_aspect": (1,1,1),
            "aspect": "equalxy"
        }
    },
    figsize=(11.0, 11.0),
    constrained_layout=True
)

line, = axs["C"].plot(xs, ys, ',', color="black", markeredgecolor=None, picker=True, pickradius=5)
axs["C"].set_title('Kinematics plot')
axs["C"].set_xlim(0.0, np.pi)
axs["C"].set_ylim(0.0, 3.0)

line2, = axs["D"].plot(de, ys, ',', color="black", markeredgecolor=None, picker=True, pickradius=5)
axs["D"].set_title('PID plot')
axs["D"].set_xlim(0.0, 120.0)
axs["D"].set_ylim(0.0, 3.0)

browser = PointBrowser()
fig.canvas.mpl_connect('pick_event', browser.on_pick)

event = 1 # Use first event to setup
browser.update_plots(event)

plt.show()